Importing necessary libraries

In [87]:
from geopy.geocoders import Nominatim
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import time
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

retrieve standard table using BeautifulSoup

In [4]:
toronto_wiki = requests.get(wikipedia_link)

In [5]:
toronto_wiki = requests.get(wikipedia_link)
soup = BeautifulSoup(toronto_wiki.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Naming Columns

In [6]:
df.columns = df.loc[0,].astype(str)
df.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [7]:
df.drop(0, inplace = True)
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Grouping Values that are repeated 

In [8]:
df1 = df.groupby(['Postcode', 'Borough']).agg(lambda col: ', '.join(col))
df1.reset_index(inplace = True)
print(df1.shape)
df1.head()

(180, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


Deleating Not Assigned Values

In [9]:
df2 = df1[df1['Borough'] != 'Not assigned']
print(df2.shape)
df2.head()

(103, 3)


,Postcode,Borough,Neighbourhood
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [10]:
df2 = df2.reset_index()
df2.head(10)

,index,Postcode,Borough,Neighbourhood
0,1,M1B,Scarborough,"Rouge, Malvern"
1,2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,4,M1G,Scarborough,Woburn
4,5,M1H,Scarborough,Cedarbrae
5,6,M1J,Scarborough,Scarborough Village
6,7,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,8,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,9,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,10,M1N,Scarborough,"Birch Cliff, Cliffside West"


Create empty latlong dataframe

In [11]:
# define the dataframe columns
column_names = ['Latitude', 'Longitude'] 

# instantiate the dataframe
LatLong = pd.DataFrame(columns=column_names)

create function to add coordinate values

In [12]:
def get_lat_long(Neighbourhood, Borough):
    
    if (df2.loc[i, 'Neighbourhood'].find(',') == -1):
         Neighbourhood = Neighbourhood
    else:
         Neighbourhood = df2.loc[i, 'Neighbourhood'][0:df2.loc[i, 'Neighbourhood'].find(',')]
    address = '{} {} Toronto'.format(Neighbourhood, Borough)

    geolocator = Nominatim(user_agent="Coursera")
    location = geolocator.geocode(address)
    try:
        latitude = location.latitude
        longitude = location.longitude
    except:
        return 0, 0
    
    
    return latitude, longitude

Add Values to dataframe

In [13]:
LatLong = pd.DataFrame(columns=column_names)
for i in range(103):
    a = get_lat_long(df2.loc[i, 'Neighbourhood'], df2.loc[i, 'Borough'])
    time.sleep(0.1)
    LatLong.loc[i,'Latitude'] = a[0]
    LatLong.loc[i,'Longitude'] = a[1]
    

In [24]:
LatLong.head(10)

,Latitude,Longitude
0,43.8049,-79.1658
1,43.7901,-79.1733
2,43.7549,-79.1978
3,43.7598,-79.2253
4,43.7565,-79.2267
5,43.7437,-79.2116
6,43.7142,-79.2711
7,43.7088,-79.296
8,43.7219,-79.2362
9,43.7021,-79.2601


Concatenate

In [28]:
df = pd.concat([df2, LatLong], axis = 1)
print(df.shape)
df.head()

(103, 6)


,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,1,M1B,Scarborough,"Rouge, Malvern",43.8049,-79.1658
1,2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7901,-79.1733
2,3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7549,-79.1978
3,4,M1G,Scarborough,Woburn,43.7598,-79.2253
4,5,M1H,Scarborough,Cedarbrae,43.7565,-79.2267


In [70]:
df = df[df['Latitude'] != 0]
df.reset_index(inplace = True)
df.shape

(81, 7)

In [31]:
CLIENT_ID = 'NWRCMJFRU0M4COYJCW3QRRY3ZCZ4GDWGNWYVW1UUP0UBWQHQ' # your Foursquare ID
CLIENT_SECRET = 'KQ1NDSG2T4E2KNRSQLEJ2C2ABO3CUQI4CTNGNEGF4M1PUPJQ' # your Foursquare Secret
VERSION = '20180605'
radius = 500
LIMIT = 100

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
The 

In [35]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.804930,-79.165837,Dean Park,43.804364,-79.169159,Park
1,"Rouge, Malvern",43.804930,-79.165837,Rexall I.D.A. Pan Pharmacy,43.804081,-79.169654,Pharmacy
2,"Rouge, Malvern",43.804930,-79.165837,Paul's Breakfast & Burgers,43.803835,-79.169825,Fast Food Restaurant
3,"Highland Creek, Rouge Hill, Port Union",43.790117,-79.173334,Highland Creek,43.790281,-79.173703,Neighborhood
4,"Highland Creek, Rouge Hill, Port Union",43.790117,-79.173334,Rapid Tac Roofing & Metal Works,43.788898,-79.173198,Construction & Landscaping


In [36]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",39,39,39,39,39,39
Agincourt,10,10,10,10,10,10
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",32,32,32,32,32,32
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",4,4,4,4,4,4
Bayview Village,13,13,13,13,13,13
"Bedford Park, Lawrence Manor East",2,2,2,2,2,2
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",8,8,8,8,8,8


In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,...,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.0,0.000000,0.00000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.031250,0.000000,0.031250,0.000000,0.000000,0.0,0.000000,0.00000
3,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
4,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
5,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
6,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
7,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
8,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
9,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,0.0,0.000000,0.00000


In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Italian Restaurant,Breakfast Spot,Theater,Bar,Ice Cream Shop,Mexican Restaurant,Liquor Store,Furniture / Home Store,Dessert Shop
1,Agincourt,Chinese Restaurant,Hong Kong Restaurant,Cantonese Restaurant,Coffee Shop,Vietnamese Restaurant,Korean Restaurant,Train Station,Asian Restaurant,Flea Market,Fish & Chips Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Bakery,Ice Cream Shop,Chinese Restaurant,Fast Food Restaurant,Liquor Store,Movie Theater,Spa,Supermarket,Clothing Store,Taco Place
3,"Alderwood, Long Branch",Pizza Place,Pool,Pharmacy,Skating Rink,Sandwich Place,Coffee Shop,Gym,Dance Studio,Pub,Falafel Restaurant
4,"Bathurst Manor, Downsview North, Wilson Heights",Baseball Field,Playground,Convenience Store,Park,Women's Store,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food


In [71]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 4, 1, 1, 3, 4, 1])

In [84]:
toronto_merged.columns[4] = ['Neighbourhood']

TypeError: Index does not support mutable operations

In [85]:
toronto_merged = df.loc[0:79,:]
toronto_merged = toronto_merged.rename(columns={'Neighbourhood': 'Neighborhood'})

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,level_0,index,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,1,M1B,Scarborough,"Rouge, Malvern",43.8049,-79.1658,1,Pharmacy,Park,Fast Food Restaurant,Women's Store,Farmers Market,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
1,1,2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7901,-79.1733,1,Construction & Landscaping,Women's Store,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop
2,2,3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7549,-79.1978,1,Train Station,Storage Facility,Moving Target,Women's Store,Fast Food Restaurant,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
3,3,4,M1G,Scarborough,Woburn,43.7598,-79.2253,1,Fast Food Restaurant,Grocery Store,Coffee Shop,Bank,Beer Store,Gym,Toy / Game Store,Paper / Office Supplies Store,Pharmacy,Furniture / Home Store
4,4,5,M1H,Scarborough,Cedarbrae,43.7565,-79.2267,4,Fast Food Restaurant,Furniture / Home Store,Coffee Shop,Clothing Store,Gym,Discount Store,Park,Paper / Office Supplies Store,Optical Shop,Shopping Mall


In [91]:
map_clusters = folium.Map(location=[43.761539, -79.411079], zoom_start=11)
map_clusters

In [92]:
#map_clusters = folium.Map(location=[43.761539, -79.411079], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
#map_clusters

In [93]:
map_clusters